In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
from KuhnPoker import KuhnPokerModel, KuhnPokerInfoSet, Card
from TreeVisualizer import plot_trees

import numpy as np
import pickle
import ipywidgets as widgets
from IPython.display import display


np.set_printoptions(precision=3)

In [10]:
with open(f'debug/tree_snapshots.pkl', 'rb') as file:
    trees_hist = pickle.load(file)

In [11]:
trees_hist

[]

In [5]:
slider = widgets.IntSlider(min=0, max=len(trees_hist)-1, step=1, description='Hist Index')

def update_plot(index):
    plot_trees(trees_hist[index])
    
widgets.interactive(update_plot, index=slider)

TraitError: setting max < min